# Machine Learning with Broad Stock Market Timeseries - a SARIMA Rollercoaster

<a href="" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<!-- @import "[TOC]" {cmd="toc" depthFrom=1 depthTo=6 orderedList=false} -->

![]()


## Prepare your Environment

Have a jupyter environment ready, and `pip install` these libraries:
- numpy
- pandas
- yfinance

You'll need access to [analysis_utils](./analysis_utils.py) library for common functions.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

import dotenv
%load_ext dotenv

import warnings
warnings.filterwarnings("ignore")

IS_KAGGLE = os.getenv('IS_KAGGLE', 'True') == 'True'

if IS_KAGGLE:
    # Kaggle confgs
    print('Running in Kaggle...')
    %pip install yfinance
    %pip install statsmodels
    %pip install seaborn
    %pip install itertools
    %pip install scikit-learn

    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
else:
    print('Running Local...')

import yfinance as yf
from analysis_utils import load_ticker_prices_ts_df, load_ticker_ts_df

os.getcwd()

C:\Users\adamd\AppData\Local\Temp\ipykernel_22756\3477214583.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Running Local...


'c:\\Users\\adamd\\workspace\\quant_research'

# Data Collection and Preprocessing

Collect market time series data (like stock prices, trading volumes, etc.).
Clean the data to handle missing values, outliers, or anomalies.
Ensure the data is in a time series format, typically with a timestamp.

In [2]:
START_DATE = "2013-01-01"
END_DATE = "2023-12-31"
DATA_DIR = "data"

os.makedirs(DATA_DIR, exist_ok=True)

tickers = {
    "SP5": "^GSPC",  # S&P 500
    # "VTW": "VT",  # Vanguard Total World Stock ETF
    # "VIX": "^VIX",  # CBOE Volatility Index
    # "10YTT": "^TNX",  # 10 Year Treasury Note Yield
    # "VEU": "VEU",  # Vanguard FTSE All-World ex-US ETF
}

dataframes = {}
for symbol, ticker in tickers.items():
    cached_file_path = f"{DATA_DIR}/{symbol}-{END_DATE}.pkl"

    try:
        if os.path.exists(cached_file_path):
            df = pd.read_pickle(cached_file_path)
        else:
            df = yf.download(
                ticker, start=START_DATE, end=END_DATE, progress=False, interval="1d"
            )
            assert len(df) > 0
            df.to_pickle(cached_file_path)
        dataframes[symbol] = df
    except Exception as e:
        print(f"Error with {symbol}: {e}")

sp500_df = dataframes.get("SP5")

sp500_df.tail(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-22,4753.919922,4772.939941,4736.770020,4754.629883,4754.629883,3046770000
2023-12-26,4758.859863,4784.720215,4758.450195,4774.750000,4774.750000,2513910000
2023-12-27,4773.450195,4785.390137,4768.899902,4781.580078,4781.580078,2748450000
2023-12-28,4786.439941,4793.299805,4780.979980,4783.350098,4783.350098,2698860000
2023-12-29,4782.879883,4788.430176,4751.990234,4769.830078,4769.830078,3126060000


In [3]:
from tensorflow.keras.layers import (
    SpatialDropout1D,
    Dense,
    Conv1D,
    Layer,
    Normalization,
    Add,
    Input,
    Lambda,
)
from tensorflow.keras import Model


class TCNBlock(Layer):
    """
    TCN Residual Block that uses zero-padding to maintain `steps` value of the ouput equal to the one in the input.
    Residual Block is obtained by stacking togeather (2x) the following:
        - 1D Dilated Convolution
        - ReLu
        - Spatial Dropout
    And adding the input after trasnforming it with a 1x1 Conv
    forked and extended from: https://github.com/albertogaspar/dts/blob/master/dts/models/TCN.py
    """

    def __init__(
        self,
        filters=1,
        kernel_size=2,
        dilation_rate=None,
        kernel_initializer="glorot_normal",
        bias_initializer="glorot_normal",
        kernel_regularizer=None,
        bias_regularizer=None,
        use_bias=False,
        dropout_rate=0.0,
        id=None,
        **kwargs,
    ):
        """ "
        Arguments
            filters: Integer, the dimensionality of the output space
                (i.e. the number of output filters in the convolution).
            kernel_size: An integer or tuple/list of a single integer,
                specifying the length of the 1D convolution window.
            dilation_rate: an integer or tuple/list of a single integer, specifying
                the dilation rate to use for dilated convolution.
                Usually dilation rate increases exponentially with the depth of the network.
            activation: Activation function to use
                If you don't specify anything, no activation is applied
                (ie. "linear" activation: `a(x) = x`).
            use_bias: Boolean, whether the layer uses a bias vector.
            kernel_initializer: Initializer for the `kernel` weights matrix
            bias_initializer: Initializer for the bias vector
            kernel_regularizer: Regularizer function applied to the `kernel` weights matrix
            bias_regularizer: Regularizer function applied to the bias vector
                (see [regularizer](../regularizers.md)).
        # Input shape
            3D tensor with shape: `(batch, steps, n_features)`
        # Output shape
            3D tensor with shape: `(batch, steps, filters)`
        """
        super(TCNBlock, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate

        # Capture feature set from the input
        self.conv1 = Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            use_bias=use_bias,
            bias_initializer=bias_initializer,
            bias_regularizer=bias_regularizer,
            kernel_initializer=kernel_initializer,
            kernel_regularizer=kernel_regularizer,
            padding="causal",
            dilation_rate=dilation_rate,
            activation="relu",
            name=f"Conv1D_1_{id}",
        )

        # Spatial dropout is specific to convolutions by dropping an entire timewindow,
        # not to rely too heavily on specific features detected by the kernels.
        self.dropout1 = SpatialDropout1D(
            dropout_rate, trainable=True, name=f"SpatialDropout1D_1_{id}"
        )
        # Capture a higher order feature set from the previous convolution
        self.conv2 = Conv1D(
            filters=filters,
            kernel_size=kernel_size,
            use_bias=use_bias,
            bias_initializer=bias_initializer,
            bias_regularizer=bias_regularizer,
            kernel_initializer=kernel_initializer,
            kernel_regularizer=kernel_regularizer,
            padding="causal",
            dilation_rate=dilation_rate,
            activation="relu",
            name=f"Conv1D_2_{id}",
        )
        self.dropout2 = SpatialDropout1D(
            dropout_rate, trainable=True, name=f"SpatialDropout1D_2_{id}"
        )

        # The skip connection is an addition of the input to the block with the output of the second dropout layer.
        # Solves vanishing gradient, carries info from earlier layers to later layers, allowing gradients to flow across this alternative path.
        # Does not learn direct mappings, but differences (residuals) while keeping temporal context.
        # Note how it keeps dims intact with kernel 1.
        self.skip_out = Conv1D(
            filters=filters,
            kernel_size=1,
            activation="linear",
            name=f"Conv1D_skipconnection_{id}",
        )
        # This is the elementwise add for the residual connection and Conv1d 2's output
        self.residual_out = Add(name=f"residual_Add_{id}")

    def apply_block(self, inputs):
        x = self.conv1(inputs)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.dropout2(x)

        # Residual output by adding the inputs back:
        skip_out_x = self.skip_out(inputs)
        x = self.residual_out([x, skip_out_x])
        return x


def TCN(
    input_shape,
    output_horizon=1,
    num_filters=32,
    num_layers=1,
    kernel_size=2,
    dilation_rate=2,
    kernel_initializer="glorot_normal",
    bias_initializer="glorot_normal",
    kernel_regularizer=None,
    bias_regularizer=None,
    use_bias=False,
    dropout_rate=0.0,
):
    """
    Tensorflow TCN Model builder.
    forked and extended from: https://github.com/albertogaspar/dts/blob/master/dts/models/TCN.py
    see: https://www.tensorflow.org/api_docs/python/tf/keras/Model
    see: https://www.tensorflow.org/guide/keras/making_new_layers_and_models_via_subclassing#the_model_class
    see: https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L2

    :param layers: int
        Number of layers for the network. Defaults to 1 layer.
    :param filters: int
        the number of output filters in the convolution. Defaults to 32.
    :param kernel_size: int or tuple
        the length of the 1D convolution window
    :param dilation_rate: int
        the dilation rate to use for dilated convolution. Defaults to 1.
    :param output_horizon: int
        the output horizon.
    """
    x = inputs = Input(shape=input_shape)
    for i in range(num_layers):
        block = TCNBlock(
            filters=num_filters,
            kernel_size=kernel_size,
            dilation_rate=dilation_rate**i,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            use_bias=use_bias,
            dropout_rate=dropout_rate,
            id=i,
        )
        x = block.apply_block(x)
    # Selects the last timestep and predict the demand in the 1 DIM layer.
    x = Lambda(lambda x: x[:, -1:, 0], name="lambda_last_timestep")(x)
    outputs = Dense(output_horizon, name="Dense_singleoutput")(x)

    model = Model(inputs=inputs, outputs=outputs, name="TCN")
    return model

In [4]:
sp500_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [5]:
def encode_timewindows(data_df, features, target, window_size, horizon):
    """
    Create input and target windows suitable for TCN model.

    :param data: DataFrame with shape (n_samples, n_features)
    :param features: List of strings, names of the feature columns
    :param target: String, name of the target column
    :param window_size: int, length of the input sequence.
    :param horizon: int, forecasting horizon.
    :return: Array in the shape of (n_samples, n_steps, n_features)
    """
    X, y = [], []
    for i in tqdm(
        range(len(data_df) - window_size - horizon + 1), desc="Encoding Widows"
    ):
        input_window = data_df[features].iloc[i : i + window_size].values
        X.append(input_window)

        # Target window, note it predicts {horizon} steps ahead
        if horizon == 1:
            target_value = data_df[target].iloc[i + window_size]
        else:
            target_value = (
                data_df[target].iloc[i + window_size : i + window_size + horizon].values
            )
        y.append(target_value)
    return np.array(X), np.array(y)


MONTH_SINE = "month_sin"
MONTH_COS = "month_cos"


def encode_cyclics(data_df):
    """
    Encodes time cyclic features for a dataset with monthly sampling.
    Assuming we can capture the yearly periodicity by encoding the month as a wave.
    See: https://www.tensorflow.org/tutorials/structured_data/time_series#time
    :param data_df: The timeseries with a date in the format YYYY-DD-mm as index.
    :return: data_df with 2 new wave features.
    """
    months = data_df.index.month

    data_df[MONTH_SINE] = np.sin(2 * np.pi * months / 12)
    data_df[MONTH_COS] = np.cos(2 * np.pi * months / 12)
    return data_df


TARGET = "Adj Close"
FEATURES = [TARGET, "Open", "High", "Low", "Volume"]
INDEX = "Date"
WINDOW_SIZE = 1 * 5  # 1 week 8hrs trading
EXT_FEATURES = FEATURES + [MONTH_SINE, MONTH_COS]
PREDICTION_HORIZON = 1  # next 1 day


def prepare_data_and_windows(data_df, window=WINDOW_SIZE, horizon=PREDICTION_HORIZON):
    """
    Utility function to prepare the data.
    :data_df dataframe: dataframe with `window_size` months of data to predict the `window_size`+`horizon`.
    :param window_size: int, length of the input sequence
    :param horizon: int, forecasting horizon, defaults to 1
    :return: Array in the shape of (n_samples, n_steps, n_features)

    """
    normalizer = Normalization(axis=-1)

    normalizer.adapt(data_df[FEATURES])
    data_df_normalized = normalizer(data_df[FEATURES])
    data_df_normalized = pd.DataFrame(
        data_df_normalized, columns=FEATURES, index=data_df.index
    )
    data_df_normalized = encode_cyclics(data_df_normalized)
    X, y = encode_timewindows(
        data_df_normalized,
        EXT_FEATURES,
        TARGET,
        window,
        horizon,
    )
    print(
        f"FEATURES: {EXT_FEATURES}, TARGET: '{TARGET}', window: {WINDOW_SIZE}, horizon: {PREDICTION_HORIZON}"
    )
    print(
        f"Shape unencoded (including target label and superflous features): {data_df.shape}"
    )
    print(f"Shape encoded (window and selected exog features only): {X.shape}")

    return X, y, normalizer


# Yes, we are using the whole dataset not the training dataset.
# See: https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
# we will tell keras to do a validation split, it will not fit on the validation data.
X, y, normalizer = prepare_data_and_windows(sp500_df)
print(f"Label shape encoded: {y.shape}")
print(f"Data shape: {X.shape}")
print(f"First window exog normalized: {X[0,  :]}")
print(f"First window targets: {y[:1]}")

input_shape = (WINDOW_SIZE, X.shape[2])
input_shape

Encoding Widows: 100%|██████████| 2763/2763 [00:01<00:00, 1766.61it/s]

FEATURES: ['Adj Close', 'Open', 'High', 'Low', 'Volume', 'month_sin', 'month_cos'], TARGET: 'Adj Close', window: 5, horizon: 1
Shape unencoded (including target label and superflous features): (2768, 6)
Shape encoded (window and selected exog features only): (2763, 5, 7)
Label shape encoded: (2763,)
Data shape: (2763, 5, 7)
First window exog normalized: [[-1.50244939 -1.54041171 -1.50813103 -1.53370547  0.32189605  0.5
   0.8660254 ]
 [-1.5056777  -1.50205755 -1.50493467 -1.50242615 -0.07017848  0.5
   0.8660254 ]
 [-1.49816263 -1.50528634 -1.50233769 -1.49873757 -0.49650061  0.5
   0.8660254 ]
 [-1.50301039 -1.49777019 -1.50388324 -1.50126421 -0.62196624  0.5
   0.8660254 ]
 [-1.50802755 -1.50261867 -1.50869894 -1.50657332 -0.31005836  0.5
   0.8660254 ]]
First window targets: [-1.5039313]


(5, 7)

In [6]:
from tensorflow.keras.regularizers import L2
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from datetime import datetime
from sklearn.model_selection import ParameterGrid
import json

VAL_SPLIT = 0.1
EPOCHS = 300
BATCH_SIZE = 32
FILTER = 128
DROPRATE = 0.5
POOL_SIZE = 2
KERNEL_SIZE = 4
DILATION_RATE = 4
MAX_LAYERS = 4
L2_REG = 0.005
LEARN_RATE = 0.0001
MODEL_LOG_DIR = f'./logs/{datetime.now().strftime("%m%d-%H%M%S")}'
# See: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html
# See paper: https://www.mdpi.com/2076-3417/10/7/2322
GRID = {
    "num_filters": [32, 64, 128],
    "kernel_size": [2, 3, 4],
    "batch_size": [64, 128, 255],
    "epochs": [25, 50, 100, 300],
    "dilation_rate": [1, 2, 4],
    "dropout_rate": [0.1, 0.2, 0.3],
    "num_layers": [6, 5, 3],
    "l2_reg": [0.005, 0.001, 0.01],
    "learning_rate": [0.001, 0.01, 0.1],
}

print(f"Model logs for Tensorboard available here: {MODEL_LOG_DIR}")


def grid_search(X, y, param_grid=GRID, file_name="best_params.json"):
    """Runs for 3 days!!!"""

    def _create_model(hyperparams):
        model = TCN(
            input_shape=input_shape,
            output_horizon=PREDICTION_HORIZON,
            num_filters=hyperparams["num_filters"],
            kernel_size=hyperparams["kernel_size"],
            num_layers=hyperparams["num_layers"],
            dilation_rate=hyperparams["dilation_rate"],
            kernel_regularizer=L2(l2=hyperparams["l2_reg"]),
            bias_regularizer=L2(l2=hyperparams["l2_reg"]),
            dropout_rate=hyperparams["dropout_rate"],
        )
        optimizer = Adam(hyperparams["learning_rate"])
        model.compile(loss="mse", optimizer=optimizer, metrics=["mse", "mae", "mape"])
        return model

    def _save_best_params(best_params, best_loss, file_name="best_params.json"):
        with open(file_name, "w") as file:
            json.dump({"best_params": best_params, "best_loss": best_loss}, file)

    grid = list(ParameterGrid(param_grid))
    best_model = None
    best_loss = np.inf
    best_params = None

    for params in tqdm(grid, desc="Grid Search.."):
        model = _create_model(params)
        callbacks = [EarlyStopping(patience=10, monitor="val_loss")]
        history = model.fit(
            X,
            y,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_split=VAL_SPLIT,
            verbose=0,
            callbacks=callbacks,
        )
        val_loss = np.min(history.history["val_loss"])

        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model
            best_params = params
            _save_best_params(best_params, best_loss, file_name)

    return best_model, best_loss


def build_tcn(X, y, input_shape):
    print(f"X: {X.shape}, y: {y.shape}, input: {input_shape}")
    model = TCN(
        input_shape=input_shape,
        output_horizon=PREDICTION_HORIZON,
        num_filters=FILTER,
        kernel_size=KERNEL_SIZE,
        num_layers=MAX_LAYERS,
        dilation_rate=DILATION_RATE,
        kernel_regularizer=L2(l2=L2_REG),
        bias_regularizer=L2(l2=L2_REG),
    )

    # See: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
    optimizer = Adam(LEARN_RATE)
    metrics = ["mse", "mae", "mape"]
    model.compile(loss="mse", optimizer=optimizer, metrics=metrics)

    # Paper's `patience` was 50, we limited to 25 and watch the MAPE
    # see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
    callbacks = [
        EarlyStopping(patience=25, monitor="val_mape", restore_best_weights=True),
        TensorBoard(log_dir=MODEL_LOG_DIR),
    ]
    history = model.fit(
        X,
        y,
        validation_split=VAL_SPLIT,
        shuffle=False,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1,
    )
    return model, history


model, history = build_tcn(X, y, input_shape)
model.summary()

Model logs for Tensorboard available here: ./logs/0126-204333
X: (2763, 5, 7), y: (2763,), input: (5, 7)
Epoch 1/300
78/78 [==============================] - 16s 167ms/step - loss: 4.2254 - mse: 0.1528 - mae: 0.2589 - mape: 152.7421 - val_loss: 3.7808 - val_mse: 0.1359 - val_mae: 0.3511 - val_mape: 24.0082
Epoch 2/300
78/78 [==============================] - 11s 143ms/step - loss: 3.3534 - mse: 0.0413 - mae: 0.1524 - mape: 78.3344 - val_loss: 3.0064 - val_mse: 0.0114 - val_mae: 0.0882 - val_mape: 6.1925
Epoch 3/300
78/78 [==============================] - 12s 150ms/step - loss: 2.7695 - mse: 0.0344 - mae: 0.1371 - mape: 60.1703 - val_loss: 2.5305 - val_mse: 0.0435 - val_mae: 0.1888 - val_mape: 13.1734
Epoch 4/300
78/78 [==============================] - 13s 173ms/step - loss: 2.2947 - mse: 0.0136 - mae: 0.0894 - mape: 52.6781 - val_loss: 2.1178 - val_mse: 0.0342 - val_mae: 0.1710 - val_mape: 12.0132
Epoch 5/300
78/78 [==============================] - 14s 181ms/step - loss: 1.9313 - ms

KeyboardInterrupt: 

In [ ]:
VAL_SIZE = round(len(X) * VAL_SPLIT)

train_data = X[:-VAL_SIZE]
test_data = X[-VAL_SIZE:]
ytrain_data = y[:-VAL_SIZE]
ytest_data = y[-VAL_SIZE:]
print(ytest_data.shape)
print(ytest_data)

y_pred = model.predict(train_data)
yt_pred = model.predict(test_data)

print(yt_pred.shape)
print(yt_pred.flatten())

In [ ]:
from sklearn.metrics import (
    r2_score,
    mean_absolute_percentage_error,
    mean_absolute_error,
    mean_squared_error
)

def wape(actual_values, forecast_values):
    """
    Calculate Weighted Absolute Percentage Error (WAPE).
    see:  https://www.obviously.ai/post/introducing-wape
    Parameters:
    - actual_values: numpy array, actual values
    - forecast_values: numpy array, predicted values

    Returns:
    - wape: float, WAPE value
    """
    total_actual = np.sum(actual_values)
    if total_actual == 0:
        return np.nan
    absolute_errors = np.abs(actual_values - forecast_values)
    wape = np.sum(absolute_errors) / total_actual
    return wape

print(f"shapes y_pred: {y_pred.shape} and yt_pred: {yt_pred.shape}")
mae_train = mean_absolute_error(ytrain_data, y_pred)
mae_test = mean_absolute_error(ytest_data, yt_pred)
mse_train = mean_squared_error(ytrain_data, y_pred)
mse_test = mean_squared_error(ytest_data, yt_pred)
rmse_train = mean_squared_error(ytrain_data, y_pred, squared=False)
rmse_test = mean_squared_error(ytest_data, yt_pred, squared=False)
mape_train = mean_absolute_percentage_error(ytrain_data, y_pred) * 100
mape_test = mean_absolute_percentage_error(ytest_data, yt_pred) * 100
wape_train = wape(ytrain_data, y_pred)
wape_test = wape(ytest_data, yt_pred)
r2 = r2_score(
    ytest_data,
    yt_pred,
)

metrics_df = pd.DataFrame(
    {
        "MAE": mae_test,
        "MSE": mse_test,
        "RMSE": rmse_test,
        "MAPE": mape_test,
        "WAPE": wape_test,
        "R2": r2,
    }
)
fig, axs = plt.subplots(3, 1, figsize=(10, 10))
axs[0].plot(history.history["loss"], label="Train loss")
axs[0].plot(history.history["val_loss"], label="Validation loss")
axs[0].set_xlabel("Epochs")
axs[0].set_ylabel("Loss")
axs[0].legend()
axs[1].plot(
    history.history["mse"],
    label="Train MSE",
)

axs[1].plot(
    history.history["val_mse"],
    label="Test MSE",
)

axs[1].set_ylim((0, 0.1))

axs[1].axhline(rmse_train, color="r", linestyle="--", label="Train Sample RMSE")
axs[1].axhline(rmse_test, color="r", linestyle="-", label="Test Sample RMSE")
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Error")
axs[1].legend()
axs[2].plot(
    history.history["mape"],
    label="Train MAPE",
)
axs[2].plot(
    history.history["val_mape"],
    label="Test MAPE",
)
axs[2].axhline(
    wape_train, color="b", linestyle="--", label="Train Sample WAPE", alpha=0.5
)

axs[2].axhline(wape_test, color="b", linestyle="-", label="Test Sample WAPE", alpha=0.5)
axs[2].set_ylim((0, 40))
axs[2].set_xlabel("Epochs")
axs[2].set_ylabel("Error")
axs[2].legend()
plt.show()

metrics_df

# Conclusion




![]()

## References

- [YFinance Github](https://github.com/ranaroussi/yfinance)
- [Vanguard All World excluding US](https://investor.vanguard.com/investment-products/etfs/profile/veu)


## Github

Article here is also available on [Github]()

Kaggle notebook available [here]()


## Media

All media used (in the form of code or images) are either solely owned by me, acquired through licensing, or part of the Public Domain and granted use through Creative Commons License.

## CC Licensing and Use

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.